In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.


KeyboardInterrupt: 

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "MLTrainCPUClu"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpucluster")
except ComputeTargetException:
    print("Creating new cpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

### YOUR CODE HERE ###

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--momentum': uniform(0.6,0.99),
        '--learning_rate': loguniform(-6, -3)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py


script_params = {
       '--kernel': 'linear',
       '--penalty': 1.0,
   }

estimator = SKLearn(#source_directory=project_folder,
                       script_params=script_params,
                       compute_target=cpu_cluster,
                       entry_script='train.py',
                       pip_packages=['joblib==0.13.2']
                      )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.


hdc = HyperDriveRunConfig(estimator=estimator, 
                          hyperparameter_sampling=ps, 
                          policy=policy, 
                          primary_metric_name='best_val_acc', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=100,
                          max_concurrent_runs=4)



In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hd_run = exp.submit(hdc)
RunDetails(hd_run).show()

### YOUR CODE HERE ###

In [ ]:
import joblib
# Get your best run and save the model from that run.
#best_run, fitted_model = hd_run.get_output()
best_run = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)
joblib.dump(best_run, 'model.joblib')

### YOUR CODE HERE ###

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.data.datapath import DataPath
from azureml.core import Dataset, Datastore
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = Dataset.Tabular.from_delimited_files(path=path)
#print(ds.to_pandas_dataframe())
### YOUR CODE HERE ###

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='spearman_correlation',
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)

In [11]:
# Submit your automl run
#from azureml.core.experiment import Experiment

#ws = Workspace.from_config()

# Choose a name for the experiment and specify the project folder.
#experiment_name = 'automl-classification'

#experiment = Experiment(ws, experiment_name)

run = exp.submit(automl_config, show_output=True)

### YOUR CODE HERE ###

NameError: name 'exp' is not defined

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [5]:
# Retrieve and save your best automl model.
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)
joblib.dump(fitted_model, 'model_automl.joblib')

### YOUR CODE HERE ###


ModuleNotFoundError: No module named 'azureml.train.hyperdrive.MedianStoppingPolicy'